In [ ]:
from PIL import Image
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import hashlib
import base64
from google.colab import files  # Para o upload de arquivos no Google Colab
import io

# Gerar chaves de criptografia pública e privada
private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048, backend=default_backend())
public_key = private_key.public_key()

def embutir_texto(imagem, texto):
    """Função para embutir o texto na imagem."""
    pixels = imagem.load()
    mensagem_bin = ''.join([format(ord(char), '08b') for char in texto])
    largura, altura = imagem.size
    idx = 0
    for y in range(altura):
        for x in range(largura):
            if idx < len(mensagem_bin):
                # Verifica se o pixel tem 3 ou 4 valores (RGB ou RGBA)
                pixel = pixels[x, y]
                if len(pixel) == 3:
                    r, g, b = pixel
                elif len(pixel) == 4:
                    r, g, b, a = pixel  # Inclui o canal alfa

                # Modifica o último bit de `r` para embutir a mensagem
                r = int(format(r, '08b')[:-1] + mensagem_bin[idx], 2)

                # Atualiza o pixel com ou sem o canal alfa
                if len(pixel) == 3:
                    pixels[x, y] = (r, g, b)
                elif len(pixel) == 4:
                    pixels[x, y] = (r, g, b, a)

                idx += 1

    imagem.save("imagem_com_texto.png")
    print("Texto embutido com sucesso na imagem 'imagem_com_texto.png'.")

def recuperar_texto(imagem):
    """Função para recuperar o texto embutido na imagem."""
    pixels = imagem.load()
    largura, altura = imagem.size
    mensagem_bin = ""
    for y in range(altura):
        for x in range(largura):
            # Verifica se o pixel tem 3 ou 4 valores (RGB ou RGBA)
            pixel = pixels[x, y]
            if len(pixel) == 3:
                r, g, b = pixel
            elif len(pixel) == 4:
                r, g, b, a = pixel  # Inclui o canal alfa, se presente

            # Extrai o último bit de `r` para formar a mensagem binária
            mensagem_bin += format(r, '08b')[-1]

    # Converte a mensagem binária em texto
    mensagem = ''.join([chr(int(mensagem_bin[i:i+8], 2)) for i in range(0, len(mensagem_bin), 8)])
    print("Texto recuperado:", mensagem)
    return mensagem

def gerar_hash(imagem):
    """Função para gerar o hash de uma imagem."""
    hash_sha256 = hashlib.sha256(imagem.tobytes()).hexdigest()
    print(f"Hash da imagem: {hash_sha256}")
    return hash_sha256

def encriptar_texto(mensagem):
    """Função para encriptar o texto com chave pública."""
    mensagem_bytes = mensagem.encode('utf-8')
    texto_encriptado = public_key.encrypt(
        mensagem_bytes,
        padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA256()), algorithm=hashes.SHA256(), label=None)
    )
    print("Texto encriptado:", base64.b64encode(texto_encriptado).decode('utf-8'))
    return texto_encriptado

def decriptar_texto(texto_encriptado):
    """Função para decriptar o texto com chave privada."""
    texto_decriptado = private_key.decrypt(
        texto_encriptado,
        padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA256()), algorithm=hashes.SHA256(), label=None)
    )
    print("Texto decriptado:", texto_decriptado.decode('utf-8'))
    return texto_decriptado.decode('utf-8')

def upload_imagem():
    """Função para fazer upload de uma imagem no Colab."""
    uploaded = files.upload()
    imagem_nome = list(uploaded.keys())[0]
    imagem = Image.open(io.BytesIO(uploaded[imagem_nome]))
    return imagem, imagem_nome

def menu():
    """Menu de opções principal."""
    while True:
        print("\nMenu de Opções:")
        print("(1) Embutir texto em uma imagem usando Steganography")
        print("(2) Recuperar o texto de uma imagem alterada")
        print("(3) Gerar o hash das imagens original e alterada")
        print("(4) Encriptar a mensagem usando chave pública e privada")
        print("(5) Decriptar a mensagem encriptada")
        print("(S ou s) Sair do Menu")

        opcao = input("Digite a opção desejada: ")

        if opcao == '1':
            imagem, _ = upload_imagem()
            texto = input("Digite o texto a ser embutido: ")
            embutir_texto(imagem, texto)

        elif opcao == '2':
            imagem, _ = upload_imagem()
            recuperar_texto(imagem)

        elif opcao == '3':
            print("Faça o upload da imagem original.")
            imagem_original, _ = upload_imagem()
            print("Faça o upload da imagem alterada.")
            imagem_alterada, _ = upload_imagem()
            hash_original = gerar_hash(imagem_original)
            hash_alterada = gerar_hash(imagem_alterada)
            if hash_original != hash_alterada:
                print("As imagens são diferentes (alteração de pixels detectada).")
            else:
                print("As imagens são idênticas.")

        elif opcao == '4':
            mensagem = input("Digite a mensagem a ser encriptada: ")
            texto_encriptado = encriptar_texto(mensagem)
            print("Faça o upload da imagem para embutir o texto encriptado.")
            imagem, _ = upload_imagem()
            embutir_texto(imagem, texto_encriptado.decode('latin-1'))  # Conversão para embutir bytes

        elif opcao == '5':
            print("Faça o upload da imagem com o texto encriptado.")
            imagem, _ = upload_imagem()
            texto_encriptado_recuperado = recuperar_texto(imagem)
            decriptar_texto(texto_encriptado_recuperado.encode('latin-1'))  # Decodifica para bytes

        elif opcao.lower() == 's':
            print("Saindo da aplicação.")
            break
        else:
            print("Opção inválida. Tente novamente.")

# Executa o menu
menu()
